Team Name: **Analytics Acrobats**

Submission: **1**

In [42]:
# Import required packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
# Set seed to ensure reproducibility

seed = 2024
np.random.seed(seed)

In [44]:
# Load datasets

diet = pd.read_csv('diet.csv')
requests = pd.read_csv('requests.csv')
reviews = pd.read_csv('reviews.csv')
recipes = pd.read_csv('recipes.csv')

/var/folders/wd/38kzp1bn5139zlz7q02hldnw0000gn/T/ipykernel_11677/2005861649.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.read_csv('reviews.csv')


In [45]:
# Get overview of the datasets

print(diet.head())
print(diet.info())
print(diet.isnull().sum())

# --> One missing value in diet column

    AuthorId        Diet  Age
0  10000120E  Vegetarian   46
1   1000014D       Vegan   18
2   1000015A  Vegetarian   58
3   1000016E  Vegetarian   32
4   1000027E       Vegan   61
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271907 entries, 0 to 271906
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   AuthorId  271907 non-null  object
 1   Diet      271906 non-null  object
 2   Age       271907 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 6.2+ MB
None
AuthorId    0
Diet        1
Age         0
dtype: int64


In [46]:
# Get overview of the datasets

print(recipes.head())
print(recipes.info())
print(recipes.isnull().sum())

# --> Missing values in columns RecipeServings and RecipeYield

   RecipeId                                      Name  CookTime  PrepTime  \
0     73440              Bow Ties With Broccoli Pesto         0      1800   
1    365718                       Cashew-chutney Rice      3600       600   
2    141757  Copycat Taco Bell Nacho Fries BellGrande      3600      2700   
3    280351  Slow Cooker Jalapeno Cheddar Cheese Soup     18000      1800   
4    180505           Cool & Crisp Citrus Chiffon Pie      3600      1800   

  RecipeCategory                         RecipeIngredientQuantities  \
0          Other  c("\"6\"", "\"2\"", "\"1 1/2\"", "\"1/4\"", "\...   
1          Other  c("\"1\"", "\"3/4\"", "\"6\"", "\"5\"", "\"2\"...   
2          Other  c("\"3\"", "\"1/2\"", "\"1\"", "\"1\"", "\"3\"...   
3          Other  c("\"2\"", "\"1\"", "\"2\"", "\"2\"", "\"1\"",...   
4          Other  c("\"1\"", "\"1/4\"", "\"1/2\"", "\"1/2\"", "\...   

                               RecipeIngredientParts  Calories  FatContent  \
0  c("\"hazelnuts\"", "\"broccol

In [47]:
# Get overview of the datasets

print(reviews.head())
print(reviews.info())
print(reviews.isnull().sum())

# --> Missing values in columns Rating, Like and TestSetId

      AuthorId  RecipeId  Rating Like  TestSetId
0     2492191A     33671     2.0  NaN        1.0
1  2002019979A     92647     2.0  NaN        2.0
2      408594E    161770     NaN  NaN        3.0
3  2001625557E    108231     2.0  NaN        4.0
4  2001427116E     71109     NaN  NaN        5.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140195 entries, 0 to 140194
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   AuthorId   140195 non-null  object 
 1   RecipeId   140195 non-null  int64  
 2   Rating     77108 non-null   float64
 3   Like       97381 non-null   object 
 4   TestSetId  42814 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 5.3+ MB
None
AuthorId         0
RecipeId         0
Rating       63087
Like         42814
TestSetId    97381
dtype: int64


In [48]:
# Get overview of the datasets

print(requests.head())
print(requests.info())
print(requests.isnull().sum())

# --> No missing values

      AuthorId  RecipeId          Time  HighCalories  HighProtein  LowFat  \
0  2001012259B     73440   1799.950949           0.0  Indifferent       0   
1      437641B    365718   4201.820980           0.0          Yes       0   
2  1803340263D    141757   6299.861496           0.0  Indifferent       1   
3      854048B    280351  19801.365796           0.0          Yes       1   
4     2277685E    180505   5400.093457           0.0  Indifferent       0   

      LowSugar  HighFiber  
0            0          0  
1  Indifferent          1  
2  Indifferent          0  
3            0          1  
4            0          0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140195 entries, 0 to 140194
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   AuthorId      140195 non-null  object 
 1   RecipeId      140195 non-null  int64  
 2   Time          140195 non-null  float64
 3   HighCalories  140195 non-null  flo